In [21]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import matplotlib.pyplot as plt
import tensorflow as tf

In [63]:
#poetry_df = pd.read_csv("../data/poetry/kaggle_poem_dataset.csv").drop(['Unnamed: 0'],axis=1)
#news_df = pd.read_csv('../data/news/news_summary_more.csv')

poetry_df = pd.read_csv("../data/poetry/kaggle_poem_dataset.csv").drop(['Unnamed: 0'],axis=1)
news_df = pd.read_csv('../data/news_summary_more.csv')

In [64]:
def token_pos(doc):
    pos_list = [p[1] for p in pos_tag(word_tokenize(doc),tagset='universal')]  
    return pos_list

In [65]:
poetry_df['pos'] = poetry_df['Content'].apply(lambda row: token_pos(row), convert_dtype=False)
poetry_df['len'] = poetry_df['pos'].apply(lambda row: len(row), convert_dtype=False)
news_df['pos'] = news_df["text"].apply(lambda row: token_pos(row), convert_dtype=False)
news_df['len'] = news_df["pos"].apply(lambda row: len(row), convert_dtype=False)

In [71]:
poetry_df=poetry_df[poetry_df['len']<100]
news_df = news_df[:len(poetry_df)]

In [144]:
len(poetry_df)

2864

In [72]:
np.mean(poetry_df2['len']), np.mean(news_df2['len'])

(62.983938547486034, 68.4895251396648)

### Training the classifier: doc_embedding, deep neural network

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [160]:
#### documents are transformed into GPT-2 vectors

poetry_df['doc_embedding'] = poetry_df["Content"].apply(lambda row: sbert_model.encode(row),convert_dtype = False )
news_df['doc_embedding'] = news_df["text"].apply(lambda row: sbert_model.encode(row),convert_dtype = False )

In [161]:
X = pd.concat([poetry_df['doc_embedding'],news_df['doc_embedding']])
X = X.values
X = np.stack(X)

y = ['0']*poetry_df.shape[0] + ['1']*news_df.shape[0]
y = np.array(y).astype(np.float32)

In [187]:
import sklearn
from sklearn import model_selection
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [203]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.10, random_state=40)
X_train.shape

epochs = 5
#batch_size =2000

opt = tf.optimizers.Adam(learning_rate=0.0005)
model = Sequential()
model.add(Dense(150, input_dim=768, activation='tanh'))
model.add(Dense(50, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='tanh'))
#model.compile(loss=poem_loss, optimizer=opt, metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)

_, accuracy = model.evaluate(X_test, y_test)
#acc = np.sum(np.round(model.predict(X_test))==y_test)/tf.reshape(y_test, [-1]).shape[0]
print('Accuracy: %.2f' % (accuracy*100))


Epoch 1/5
162/162 [==============================] - 1s 1ms/step - loss: 0.1491 - accuracy: 0.9659
Epoch 2/5
162/162 [==============================] - 0s 1ms/step - loss: 0.0938 - accuracy: 0.9765
Epoch 3/5
162/162 [==============================] - 0s 1ms/step - loss: 0.0353 - accuracy: 0.9919
Epoch 4/5
162/162 [==============================] - 0s 1ms/step - loss: 0.0255 - accuracy: 0.9942
Epoch 5/5
18/18 [==============================] - 0s 1ms/step - loss: 0.0146 - accuracy: 0.9948
Accuracy: 99.48


In [204]:
model.save('classifier_model.h5')


### Test on the evaluation data 

In [205]:
model = tf.keras.models.load_model('classifier_model.h5')
test = pd.read_csv('eval_data.csv')
y_test = [0]*len(test)

In [206]:
X_test = test["poem"].apply(lambda row: sbert_model.encode(row),convert_dtype = False )
X_test = X_test.values
X_test = np.stack(X_test)

In [207]:
model.evaluate(X_test, np.array(y_test))

1/1 [==============================] - 0s 120ms/step - loss: 0.4144 - accuracy: 0.8750


[0.41441434621810913, 0.875]